But it's so boring. Also it's not full automated way.

Unfortunately, pd.read_html and requests + beautifulsoup do not allow parsing info.
Therefore, I decided to use API.

First and foremost, I registered in https://etherscan.io/ to get API key.

FREE API PLAN allows doing 5 calls per second. It's appropriate number for our goals.

API-Key is kept in .env file.But it's so boring. Also it's not full automated way.

Unfortunately, pd.read_html and requests + beautifulsoup do not allow parsing info.
Therefore, I decided to use API.

First and foremost, I registered in https://etherscan.io/ to get API key.

FREE API PLAN allows doing 5 calls per second. It's appropriate number for our goals.

API-Key is kept in .env file.

In [66]:
%load_ext autoreload
%autoreload 2
%aimport

import requests
from bs4 import BeautifulSoup
from application.load_transaction_data import load_ether_data, load_token_data
import pandas as pd
import re
from tqdm import tqdm
import random
import time

transactions_df = load_ether_data()
token_df = load_token_data()
all_tokens_df_2 = pd.read_pickle("all_tokens_df_2")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Modules to reload:
all-except-skipped

Modules to skip:



# First parsing attempt

In [ ]:
def extract_token_info(token_url):
    result_dict = {}
    try:
        result_dict['from'] =  token_url.find("span", class_='hash-tag text-truncate hash-tag-custom-from tooltip-address').text
    except Exception as e:
        result_dict['from'] = e
    try:
        result_dict['to'] = token_url.find("span", class_='hash-tag text-truncate hash-tag-custom-to tooltip-address').text
    except Exception as e:
        result_dict['to'] = e
    try:
        if token_url.find("span", 'data-toggle'=='tooltip').text != 'Balancer Poo...':
            result_dict['amount'] = token_url.find("span", 'data-toggle'=='tooltip').text
        else:
            result_dict['amount'] = token_url.find_all("span", class_='mr-1')[-1].text
            result_dict['amount'] = float(result_dict['amount'].replace(",", "_"))

    except AttributeError:
        result_dict['amount'] = token_url.find_all("span", class_='mr-1')[-1].text
        result_dict['amount'] = float(result_dict['amount'].replace(",", "_"))
    except Exception as e:
        result_dict['amount'] = e
    try:

        if token_url.find("span", 'data-toggle'=='tooltip').text != 'Balancer Poo...':
            # token1_usd = token_url.find("button", class_='u-label u-label--value u-label--primary rounded mb-n1 ml-1').text
            token1_usd = re.findall(r"\(\$.*?\)", str(token_url))[0]
            token1_usd = token1_usd.strip("()").replace(" (", "").replace("$", "")
            result_dict['usd'] = float(token1_usd.replace(",", "_"))
        else:
            result_dict['usd'] = 'empty'
    except AttributeError:
        result_dict['usd'] = 'empty'
    except Exception as e:
        result_dict['usd'] = e
    try:
        result_dict['token_hash'] = token_url.find("a").get("href").split("?")[0].split("/")[2]
    except Exception as e:
        result_dict['token_hash'] = e
    return result_dict

def extract_tokens_info(transaction_hash):
    req = requests.get(f"https://etherscan.io/tx/{transaction_hash}",
                   headers={"User-Agent": generate_user_agent()})
    content = req.content
    soup = BeautifulSoup(content)
    tokens_df = []
    bad_tokens = []
    errors = []
    all_tokens = soup.find_all("li", class_='media align-items-baseline mb-2')
    for index, tokens in enumerate(all_tokens):
        try:
            tokens_df.append(
                pd.DataFrame.from_dict(extract_token_info(all_tokens[index]), orient='index').T
            )
        except Exception as e:
            bad_tokens.append(index)
            errors.append(e)
    try:

        tokens_df = pd.concat(tokens_df)
    except ValueError:
        tokens_df = pd.DataFrame(data=[transaction_hash], columns=['Txhash'])

    # TRANSFER parsing
    transfer_text_list = []
    transfers = soup.find_all("li", class_='media align-items-baseline')
    if len(transfers) > 0:
        for transf in transfers:
            transfer_text_list.append(transf.text.replace("\xa0", ""))
        tokens_df['TRANSFER'] = pd.Series(transfer_text_list)

    tokens_df['Txhash'] = transaction_hash
    return tokens_df

all_tokens_df = []
for tx_hash in tqdm(transactions_df['Txhash'].unique().tolist()):
    try:
        time.sleep(random.uniform(0, 2))
        all_tokens_df.append(extract_tokens_info(tx_hash))
    except Exception as e:
        print(e)
        print(f"Error on {tx_hash}")
    pd.concat(all_tokens_df).to_pickle("transactions_parsed_df")
transactions_parsed_df = pd.read_pickle("transactions_parsed_df")
all_tokens_user_df = all_tokens_df[(all_tokens_df["from"] == '0x99fd1378ca799ed6772fe7bcdc9b30b389518962') |
                                     (all_tokens_df['to'] == '0x99fd1378ca799ed6772fe7bcdc9b30b389518962')]

 10%|█         | 754/7527 [19:00<2:22:56,  1.27s/it]

## Parsing token hash

In [67]:
all_token_hash = [i for i in set(all_tokens_user_df['token_hash'].unique()) if type(i) == str and len(i) == 42]
def parsing_tokens(token_hash):
    token_dict = {}
    token_dict['token_hash'] = token_hash
    req = requests.get(f"https://etherscan.io/token/{token_hash}",
                   headers={"User-Agent": generate_user_agent()})
    content = req.content
    soup = BeautifulSoup(content)
    try:
        token_dict['token_short_name'] = soup.find("span", class_='text-secondary small').text
    except Exception as e:
        token_dict['token_short_name'] = e
    try:
        token_dict['token_long_name'] = soup.find("a", class_='mb-1 mb-sm-0 u-label u-label--xs u-label--info').text
    except Exception as e:
        token_dict['token_long_name'] = e
    return token_dict

token_names_df = []
for token in tqdm(all_token_hash):
    time.sleep(random.uniform(0, 2))
    try:
        token_names_df.append(
                pd.DataFrame.from_dict(parsing_tokens(token), orient='index').T)
    except Exception as e:
        token_names_df.append(pd.DataFrame(data=[token], columns=['Txhash']))
token_names_df = pd.concat(token_names_df)
token_names_df.to_pickle("token_hash_name")

# Add token name
all_tokens_df_2 = all_tokens_df_2.merge(token_names_df[['token_hash', 'token_short_name']], how='left', on='token_hash')

100%|██████████| 241/241 [05:56<00:00,  1.48s/it]


In [162]:
from user_agent import generate_user_agent, generate_navigator
proxies = {"https": "188.170.233.113:3128"}
transaction_hash = "0xd31490b64ae0fb298449e377f1eda528224a76e0141b8b4f7fe35ca0faeba36e"
req = requests.get(f"https://etherscan.io/tx/{transaction_hash}",
                   headers={"User-Agent": generate_user_agent()},
                   # proxies=proxies
                   )
content = req.content
soup = BeautifulSoup(content)
all_tokens = soup.find_all("li", class_='media align-items-baseline mb-2')
token_url = all_tokens[9]
if token_url.find("span", 'data-toggle'=='tooltip').text != 'Balancer Poo...':
    token_url.find("span", 'data-toggle'=='tooltip').text
else:


'Balancer Poo...'

In [149]:
delete = pd.read_pickle("transactions_parsed_df")
delete

,Txhash,from,to,amount,usd,token_hash,TRANSFER
0,0x4c3b415bb98370a9ef572567a4e13b61759763c08192...,NaN,NaN,NaN,NaN,NaN,NaN
0,0xdf08f1b6048a3c151737d797c5a5da5892cff66dfdfd...,Balancer: Exchange Proxy,Balancer: ETH/USDC 50/50 #7,0.061769927338560187,24.06,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,TRANSFER 0.1 Ether From Balancer: Exchange Pro...
0,0xdf08f1b6048a3c151737d797c5a5da5892cff66dfdfd...,Balancer: ETH/USDC 50/50 #7,Balancer: Exchange Proxy,24.136325,24.12,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,TRANSFER 0.1 Ether From Balancer: Exchange Pro...
0,0xdf08f1b6048a3c151737d797c5a5da5892cff66dfdfd...,Balancer: Exchange Proxy,Balancer: PEAK/USDC/ETH 30/40/30 #2,0.022846610049600941,8.9,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,TRANSFER 0.1 Ether From Balancer: Exchange Pro...
0,0xdf08f1b6048a3c151737d797c5a5da5892cff66dfdfd...,Balancer: PEAK/USDC/ETH 30/40/30 #2,Balancer: Exchange Proxy,8.94618,8.94,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,TRANSFER 0.1 Ether From Balancer: Exchange Pro...
...,...,...,...,...,...,...,...
0,0x0b088056cf3e8bc57cbbb6fe35998f2e4c1d5b9966d9...,Balancer: renBTC/WBTC/YFV 49/49/2,"DSProxy #167,870",Balancer Poo...,list index out of range,0xdc7d39628855b6013000c9af957e6cd484beda6c,NaN
0,0x0b088056cf3e8bc57cbbb6fe35998f2e4c1d5b9966d9...,"DSProxy #167,870",0x99fd1378ca799ed6772fe7bcdc9b30b389518962,0.00128606,13.31,0xeb4c2781e4eba804ce9a9803c67d0893436bb27d,NaN
0,0x0b088056cf3e8bc57cbbb6fe35998f2e4c1d5b9966d9...,"DSProxy #167,870",0x99fd1378ca799ed6772fe7bcdc9b30b389518962,0.15680746,1627.39,0x2260fac5e5542a773aa44fbcfedf7c193bc2c599,NaN
0,0x0b088056cf3e8bc57cbbb6fe35998f2e4c1d5b9966d9...,"DSProxy #167,870",0x99fd1378ca799ed6772fe7bcdc9b30b389518962,1.907659528066759102,2.86,0x45f24baeef268bb6d63aee5129015d69702bcdfa,NaN


In [141]:
transfer_text_list = []
transfers = soup.find_all("li", class_='media align-items-baseline')
if len(transfers) > 0:
    for transf in transfers:
        transfer_text_list.append(transfers[1].text.replace("\xa0", ""))

In [142]:
transfer_text_list

['TRANSFER 6.497547074746942237 Ether From Balancer: Exchange Proxy To  0x99fd1378ca799ed6772fe7bcdc9b30b389518962',
 'TRANSFER 6.497547074746942237 Ether From Balancer: Exchange Proxy To  0x99fd1378ca799ed6772fe7bcdc9b30b389518962']

In [28]:
transaction_actions = soup.find_all("div", class_='media-body')
transaction_action = transaction_actions[0]
transaction_action_name = transaction_action.find("span", class_='text-secondary mr-1 d-inline-block').text
transaction_action_title = transaction_action.find("span", class_='mr-1 d-inline-block').get("title")
transaction_action_amount = transaction_action.find("span", class_='mr-1 d-inline-block').text
transaction_action_token_name = transaction_action.find("a", target='_parent').text

AttributeError: 'NoneType' object has no attribute 'text'